In [1]:
#importing libraries
import numpy as np
import matplotlib.pyplot as plot
import pandas as pd


In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first='true')

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
independent=dataset[['Age','EstimatedSalary','Gender_Male']]
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [7]:
dependent=dataset[['Purchased']]
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [8]:
# splitting training and test data 
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

C:\Users\sindhiya maria\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import pickle

param_grid={'criterion':['gini', 'entropy', 'log_loss'],
            'max_features':['auto','sqrt','log2'],'n_estimators':[10,100]}


grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(x_train, y_train) 


# Save the trained model to a file
filename = 'finalized_model_random_forest_grid_classification.sav'
with open(filename, 'wb') as file:
    pickle.dump(grid, file)

print("Model saved successfully.")

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Users\sindhiya maria\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.88687676 0.89874115 0.87184388 0.89903073 0.88372002 0.89521674
 0.87147282 0.89486224 0.84757684 0.89147327 0.8791621  0.89521674
        nan        nan        nan        nan        nan        nan]
  warnings.warn(
C:\Users\sindhiya maria\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


Model saved successfully.


In [11]:
# print best parameter after tuning
re=grid.cv_results_
grid_predictions = grid.predict(x_test)


from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,grid_predictions)

# print classification report 
from sklearn.metrics  import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [12]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'gini', 'max_features': 'sqrt', 'n_estimators': 100}: 0.9256751954513147


In [13]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[79  6]
 [ 4 45]]


In [14]:
print("The Report:\n",clf_report)

The Report:
               precision    recall  f1-score   support

           0       0.95      0.93      0.94        85
           1       0.88      0.92      0.90        49

    accuracy                           0.93       134
   macro avg       0.92      0.92      0.92       134
weighted avg       0.93      0.93      0.93       134



In [15]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9625450180072029

In [16]:
table =pd.DataFrame.from_dict(re)

In [17]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.021799,0.001029,0.003001,0.000632,gini,auto,10,"{'criterion': 'gini', 'max_features': 'auto', ...",0.847141,0.868752,0.869709,0.925272,0.923510,0.886877,0.031681,7
1,0.167213,0.007300,0.012077,0.001391,gini,auto,100,"{'criterion': 'gini', 'max_features': 'auto', ...",0.847141,0.907177,0.851527,0.944161,0.943699,0.898741,0.042537,2
2,0.024368,0.008919,0.002885,0.000890,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.847141,0.811321,0.851527,0.944161,0.905069,0.871844,0.046946,10
3,0.160534,0.004629,0.010914,0.000905,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.867478,0.907177,0.851527,0.925272,0.943699,0.899031,0.034648,1
4,0.021297,0.004144,0.002461,0.000430,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.867478,0.866968,0.851527,0.870362,0.962264,0.883720,0.039821,8
5,0.168379,0.007843,0.011898,0.000689,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.867478,0.888107,0.851527,0.925272,0.943699,0.895217,0.034591,3
6,0.019409,0.001562,0.003065,0.000131,entropy,auto,10,"{'criterion': 'entropy', 'max_features': 'auto...",0.847141,0.828959,0.832483,0.925272,0.923510,0.871473,0.043639,11
7,0.163250,0.006972,0.011175,0.000525,entropy,auto,100,"{'criterion': 'entropy', 'max_features': 'auto...",0.847141,0.888107,0.851527,0.925272,0.962264,0.894862,0.043974,5
8,0.018877,0.001498,0.002600,0.000490,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.782557,0.849057,0.814409,0.886792,0.905069,0.847577,0.045107,12
9,0.161602,0.003438,0.011249,0.000578,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.867478,0.870047,0.851527,0.925272,0.943041,0.891473,0.035867,6


In [1]:
age_input=float(input("Age:"))

Gender_male_input=int(input("Gender_Male 0 or 1:"))

EstimatedSalary_input=int(input("EstimatedSalary:"))


Age:46
Gender_Male 0 or 1:0
EstimatedSalary:41000


In [3]:
import pickle
loaded_model=pickle.load(open("finalized_model_random_forest_grid_classification.sav",'rb'))
Future_Prediction=loaded_model.predict([[age_input,Gender_male_input,EstimatedSalary_input]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))

C:\Users\sindhiya maria\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Future_Prediction=[1]
